In [2]:
# Let's start by importing everything we need.
import os
# To not have numpy start parallelizing on its own
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import matplotlib.pyplot as plt

import petitRADTRANS as prt
from petitRADTRANS import physical_constants as cst

# Import the class used to set up the retrieval.
from petitRADTRANS.retrieval import Retrieval,RetrievalConfig
# Import Prior functions, if necessary.
from petitRADTRANS.retrieval.utils import gaussian_prior, inverse_gamma_prior
# Import atmospheric model function
from petitRADTRANS.retrieval.models import emission_model_diseq, gradient_profile_emission, interpolated_profile_emission

In [5]:
# Define the pRT run setup
retrieval_config = RetrievalConfig(
    retrieval_name="HD984B_test", # give a useful name for your retrieval
    run_mode="retrieve",  # 'retrieve' to run, or 'evaluate' to make plots
    amr=True,  # adaptive mesh refinement, slower if True
    scattering_in_emission=True  #  add scattering for emission spectra clouds
)

In [6]:
import petitRADTRANS # need to get the name for the example data
path_to_data = "./"

retrieval_config.add_data(
    'NIRSPEC_G395H_HPF',
    path_to_data+"hd984B_nirspec_nonans.txt",
    data_resolution=5000,
    model_resolution=10000,
    model_generating_function = emission_model_diseq,
    resample=True,
    filters=True,
    radvel=True,
    line_opacity_mode='lbl'
)

In [7]:
# Add parameters, and priors for free parameters

# This run uses the model of Molliere (2020) for HR8799e
# The lambda function provide uniform priors

# Distance to the planet in cm
retrieval_config.add_parameter(
    name='D_pl',
    free=False,
    value=41.2925 * cst.pc
)

# Log of the surface gravity in cgs units.
retrieval_config.add_parameter(
    'log_g',
    True,
    transform_prior_cube_coordinate = lambda x : 2 + 3.5 * x
)

# Planet mass in mj
retrieval_config.add_parameter(
    'mass',
    True,
    transform_prior_cube_coordinate=lambda x : gaussian_prior(x, 61, 4) * cst.m_jup
)

# hpf nirspec hyperparameters
retrieval_config.add_parameter(
    'NIRSPEC_G395H_HPF_nodes',
    False,
    value=60
)

retrieval_config.add_parameter(
    'NIRSPEC_G395H_HPF_radvel',
    True,
    transform_prior_cube_coordinate=lambda x : ( -5 + 5 * x)
)

retrieval_config.add_parameter(
    'NIRSPEC_G395H_HPF_R_slope',
    True,
    transform_prior_cube_coordinate=lambda x : ( 100 + 900 * x)
)

retrieval_config.add_parameter(
    'NIRSPEC_G395H_HPF_R_int',
    True,
    transform_prior_cube_coordinate=lambda x : ( 200 + 1800 * x)
)




# Temperature in Kelvin
retrieval_config.add_parameter(
    'T_int',
    True,
    transform_prior_cube_coordinate=lambda x : 300.0 + 2000.0 * x,
    value=0.0
)

# Spline temperature structure parameters. T1 < T2 < T3
# As these priors depend on each other, they are implemented in the model function
retrieval_config.add_parameter(
    'T3',
    True,
    transform_prior_cube_coordinate=lambda x : x,
    value=0.0
)
retrieval_config.add_parameter(
    'T2',
    True,
    transform_prior_cube_coordinate=lambda x : x,
    value=0.0
)
retrieval_config.add_parameter(
    'T1',
    True,
    transform_prior_cube_coordinate=lambda x : x
)
# Optical depth model
# power law index in tau = delta * press_cgs**alpha
retrieval_config.add_parameter(
    'alpha',
    True,
    transform_prior_cube_coordinate=lambda x :1.0 + x
)
# proportionality factor in tau = delta * press_cgs**alpha
retrieval_config.add_parameter(
    'log_delta',
    True,
    transform_prior_cube_coordinate=lambda x : x
)
# Chemistry
# A 'free retrieval' would have each line species as a parameter
# Using a (dis)equilibrium model, we only supply bulk parameters.
# Carbon quench pressure
retrieval_config.add_parameter(
    'log_pquench',
    True,
    transform_prior_cube_coordinate=lambda x : -6.0 + 9.0 * x
    )
# Metallicity [Fe/H]
retrieval_config.add_parameter(
    'Fe/H',
    True,
    transform_prior_cube_coordinate=lambda x : -1.5 + 3.0 * x
)
# C/O ratio
retrieval_config.add_parameter(
    'C/O',
    True,
    transform_prior_cube_coordinate=lambda x : 0.1+1.5*x
)
# Clouds
# Based on an Ackermann-Marley (2001) cloud model
# Width of particle size distribution
retrieval_config.add_parameter(
    'sigma_lnorm',
    True,
    transform_prior_cube_coordinate=lambda x : 1.05 + 1.95 * x
)
# Vertical mixing parameters
retrieval_config.add_parameter(
    'log_kzz',
    True,
    transform_prior_cube_coordinate=lambda x : 5.0 + 8.0 * x
)
# Sedimentation parameter
retrieval_config.add_parameter(
    'fsed',
    True,
    transform_prior_cube_coordinate=lambda x : 1.0 + 10.0 * x
)


In [8]:
# Define opacity species to be included

retrieval_config.set_rayleigh_species(['H2', 'He'])
retrieval_config.set_continuum_opacities(['H2-H2', 'H2-He'])
retrieval_config.set_line_species(
    [
        # '1H2-16O__POKAZATEL.R1e6_0.3-28mu',
        # '12C-16O__HITEMP',
        # '13C-16O__HITRAN',
        'H2O',
        'CO',
        # '13CO',
        # '13C16O'
        # '12C-17O'
        # 'H2O__POKAZATEL.R1e6',
        # 'CO-NatAbund__HITEMP.R1e6',
        'CH4',
        'CO2',
        'HCN',
        'H2S',
        # 'FeH.R1e6',
        # 'NH3',
        # 'PH3',
        # 'Na',
        # 'K',
        # 'TiO',
        # 'VO',
        # 'SiO'
    ],
    eq = True
)
retrieval_config.add_cloud_species('Fe(s)_crystalline__DHS', eq=True, abund_lim=(-3.5, 1.0))
retrieval_config.add_cloud_species('MgSiO3(s)_crystalline__DHS', eq=True, abund_lim=(-3.5, 1.0))



In [9]:
for specie in retrieval_config.cloud_species:
    
    retrieval_config.add_parameter(
        'eq_scaling_'+specie.split('_')[0],
        True,
        transform_prior_cube_coordinate=lambda x : -3.5 + 4.5 * x
    )

In [11]:
# Before we run the retrieval, let's set up plotting.

# Define what to put into corner plot if run_mode == 'evaluate'
# retrieval_config.parameters['planet_radius'].plot_in_corner = True
# retrieval_config.parameters['planet_radius'].corner_label = r'$R_{\rm P}$ ($\rm R_{Jup}$)'
# retrieval_config.parameters['planet_radius'].corner_transform = lambda x : x / cst.r_jup_mean
retrieval_config.parameters['mass'].plot_in_corner = True
retrieval_config.parameters['mass'].corner_label = r'$M_{\rm P}$ ($\rm M_{Jup}$)'
retrieval_config.parameters['mass'].corner_transform = lambda x : x / cst.m_jup
retrieval_config.parameters['log_g'].plot_in_corner = True
retrieval_config.parameters['log_g'].corner_ranges = [2., 5.]
retrieval_config.parameters['log_g'].corner_label = "log g"
retrieval_config.parameters['fsed'].plot_in_corner = True
retrieval_config.parameters['log_kzz'].plot_in_corner = True
retrieval_config.parameters['log_kzz'].corner_label = "log Kzz"
retrieval_config.parameters['C/O'].plot_in_corner = True
retrieval_config.parameters['Fe/H'].plot_in_corner = True
retrieval_config.parameters['log_pquench'].plot_in_corner = True
retrieval_config.parameters['log_pquench'].corner_label = "log pquench"

for spec in retrieval_config.cloud_species:
    cname = spec.split('_')[0]
    retrieval_config.parameters['eq_scaling_' + cname].plot_in_corner = True
    retrieval_config.parameters['eq_scaling_' + cname].corner_label = cname

# Define axis properties of spectral plot if run_mode == 'evaluate'
retrieval_config.plot_kwargs["spec_xlabel"] = 'Wavelength [micron]'

retrieval_config.plot_kwargs["spec_ylabel"] = "Flux [W/m2/micron]"
retrieval_config.plot_kwargs["y_axis_scaling"] = 1.0
retrieval_config.plot_kwargs["xscale"] = 'log'
retrieval_config.plot_kwargs["yscale"] = 'linear'
retrieval_config.plot_kwargs["resolution"] = 5000.0  # maximum resolution, will rebin the data
retrieval_config.plot_kwargs["nsample"] = 10  # if we want a plot with many sampled spectra

# Define from which observation object to take P-T in evaluation mode (if run_mode == 'evaluate'), add PT-envelope plotting options
retrieval_config.plot_kwargs["take_PTs_from"] = 'NIRSPEC_G395H_HPF'
retrieval_config.plot_kwargs["temp_limits"] = [150, 3000]
retrieval_config.plot_kwargs["press_limits"] = [1e2, 1e-5]

In [12]:
retrieval = Retrieval(
    retrieval_config,
    output_directory="./",
    evaluate_sample_spectra=False,
    test_plotting=False
)



Setting up Radtrans object for data 'NIRSPEC_G395H_HPF'...
Setting up AMR pressure grid.
Loading Radtrans opacities...
 Loading line opacities of species 'H2O' from file '/Users/wbalmer/data/prt3/input_data/opacities/lines/line_by_line/H2O/1H2-16O/1H2-16O__POKAZATEL.R1e6_0.3-28mu.xsec.petitRADTRANS.h5'... Done.
 Loading line opacities of species 'CO' from file '/Users/wbalmer/data/prt3/input_data/opacities/lines/line_by_line/CO/12C-16O/12C-16O__HITEMP.R1e6_0.3-28mu.xsec.petitRADTRANS.h5'... Done.
 Loading line opacities of species 'CH4' from file '/Users/wbalmer/data/prt3/input_data/opacities/lines/line_by_line/CH4/12C-1H4/12C-1H4__Hargreaves.R1e6_0.3-28mu.xsec.petitRADTRANS.h5'... Done.
 Loading line opacities of species 'CO2' from file '/Users/wbalmer/data/prt3/input_data/opacities/lines/line_by_line/CO2/12C-16O2/12C-16O2__HITEMP.R1e6_0.3-28mu.xsec.petitRADTRANS.h5'... Done.
 Loading line opacities of species 'HCN' from file '/Users/wbalmer/data/prt3/input_data/opacities/lines/line_b

In [ ]:
run_retrieval = True

if run_retrieval:
    retrieval.run(
        n_live_points=30,
        sampling_efficiency=0.5,
        const_efficiency_mode=True,
        resume=False,
        seed=-1  # ⚠️ seed should be removed or set to -1 in a real retrieval, it is added here for reproducibility
    )

Starting retrieval HD984B_test
Testing data 'NIRSPEC_G395H_HPF':
 wavelengths:
  OK (no NaN, infinite, or negative value detected)
 spectrum:
 uncertainties:
Testing model function for data 'NIRSPEC_G395H_HPF'...
Loading chemical equilibrium chemistry table from file '/Users/wbalmer/data/prt3/input_data/pre_calculated_chemistry/equilibrium_chemistry/equilibrium_chemistry.chemtable.petitRADTRANS.h5'... 

/Users/wbalmer/petitRADTRANS/petitRADTRANS/retrieval/retrieval.py:227: UserWarning: Negative value detected (1515 / 3128)
Make sure that this makes sense for the data set you are considering.
  warnings.warn(f"Negative value detected ({np.nonzero(np.less(data, 0))[0].size} / {data.size})\n"
/Users/wbalmer/petitRADTRANS/petitRADTRANS/retrieval/retrieval.py:191: UserWarning: Data may not be suitable for retrievals due to invalid values
  warnings.warn("Data may not be suitable for retrievals due to invalid values")


Done.


/Users/wbalmer/petitRADTRANS/petitRADTRANS/radtrans.py:408: UserWarning: setting a Radtrans property directly is not recommended
Create a new Radtrans instance (recommended) or re-do all the setup steps necessary for the modification to be taken into account
  warnings.warn(self.__property_setting_warning_message)


No errors detected in the model functions!
Starting retrieval: HD984B_test

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =   30
 dimensionality =   19
 running in constant efficiency mode
 *****************************************************
 Starting MultiNest
 generating live points


/Users/wbalmer/petitRADTRANS/petitRADTRANS/retrieval/retrieval.py:2275: UserWarning: Sampling efficiency should be <= 0.1 (0.05 recommended) if you're using constant efficiency mode!
  warnings.warn(


 live points generated, starting sampling


In [ ]:
if run_retrieval:
    retrieval.plot_all(contribution=True)